In [51]:
# Import our dependencies
import pandas as pd
from sklearn.cluster import KMeans
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,OneHotEncoder
import tensorflow as tf
import plotly.express as px
import hvplot.pandas

import warnings
warnings.filterwarnings('ignore')

In [52]:
file_path = "../instacart-data/order_products__train.csv"
orders_train_df = pd.read_csv(file_path)
orders_train_df.head()

file_path = "../instacart-data/order_products__prior.csv"
orders_prior_df = pd.read_csv(file_path)
orders_prior_df.head()

,order_id,product_id,add_to_cart_order,reordered
0,2,33120,1,1
1,2,28985,2,1
2,2,9327,3,0
3,2,45918,4,1
4,2,30035,5,0


In [53]:
file_path = "../instacart-data/products.csv"
products_df = pd.read_csv(file_path)
products_df.head()

,product_id,product_name,aisle_id,department_id
0,1,Chocolate Sandwich Cookies,61,19
1,2,All-Seasons Salt,104,13
2,3,Robust Golden Unsweetened Oolong Tea,94,7
3,4,Smart Ones Classic Favorites Mini Rigatoni Wit...,38,1
4,5,Green Chile Anytime Sauce,5,13


In [54]:
#orders_df = orders_prior_df.merge(orders_train_df, how="outer")
orders_df = orders_train_df.merge(products_df, how="outer", on="product_id")
orders_df.head()

,order_id,product_id,add_to_cart_order,reordered,product_name,aisle_id,department_id
0,1.0,49302,1.0,1.0,Bulgarian Yogurt,120,16
1,816049.0,49302,7.0,1.0,Bulgarian Yogurt,120,16
2,1242203.0,49302,1.0,1.0,Bulgarian Yogurt,120,16
3,1383349.0,49302,11.0,1.0,Bulgarian Yogurt,120,16
4,1787378.0,49302,8.0,0.0,Bulgarian Yogurt,120,16


In [55]:
orders_df.dropna(inplace=True)
orders_df.drop(columns=["product_name"], inplace=True)

In [56]:
#orders_df.dropna(inplace=True) dropping order_id will lose data for reorder
#orders_df.drop(columns=["order_id"], inplace=True)
#orders_df

In [57]:
orders_df = orders_df.drop_duplicates()
orders_df.to_csv("UniqueProducts.csv", index=None)
orders_df

,order_id,product_id,add_to_cart_order,reordered,aisle_id,department_id
0,1.0,49302,1.0,1.0,120,16
1,816049.0,49302,7.0,1.0,120,16
2,1242203.0,49302,1.0,1.0,120,16
3,1383349.0,49302,11.0,1.0,120,16
4,1787378.0,49302,8.0,0.0,120,16
...,...,...,...,...,...,...
1384612,3420011.0,1528,12.0,0.0,97,13
1384613,3420084.0,47935,20.0,0.0,73,11
1384614,3420084.0,9491,21.0,0.0,25,11
1384615,3420088.0,16380,12.0,0.0,97,13


In [58]:
file_path = "Reordered_prodcount.csv"
reordprod_df = pd.read_csv(file_path)
reordprod_df.head()

,product_id,reord_no.
0,36112,3
1,48775,302
2,21849,36
3,20691,15
4,45445,128


In [59]:
orders_df = orders_df.merge(reordprod_df, how="left", on="product_id")
orders_df

,order_id,product_id,add_to_cart_order,reordered,aisle_id,department_id,reord_no.
0,1.0,49302,1.0,1.0,120,16,7.0
1,816049.0,49302,7.0,1.0,120,16,7.0
2,1242203.0,49302,1.0,1.0,120,16,7.0
3,1383349.0,49302,11.0,1.0,120,16,7.0
4,1787378.0,49302,8.0,0.0,120,16,7.0
...,...,...,...,...,...,...,...
1384612,3420011.0,1528,12.0,0.0,97,13,NaN
1384613,3420084.0,47935,20.0,0.0,73,11,NaN
1384614,3420084.0,9491,21.0,0.0,25,11,NaN
1384615,3420088.0,16380,12.0,0.0,97,13,NaN


In [60]:
orders_df["reord_no."] = orders_df["reord_no."].fillna(0)
orders_df

,order_id,product_id,add_to_cart_order,reordered,aisle_id,department_id,reord_no.
0,1.0,49302,1.0,1.0,120,16,7.0
1,816049.0,49302,7.0,1.0,120,16,7.0
2,1242203.0,49302,1.0,1.0,120,16,7.0
3,1383349.0,49302,11.0,1.0,120,16,7.0
4,1787378.0,49302,8.0,0.0,120,16,7.0
...,...,...,...,...,...,...,...
1384612,3420011.0,1528,12.0,0.0,97,13,0.0
1384613,3420084.0,47935,20.0,0.0,73,11,0.0
1384614,3420084.0,9491,21.0,0.0,25,11,0.0
1384615,3420088.0,16380,12.0,0.0,97,13,0.0


In [62]:
orders_df.drop(columns=["order_id"], inplace=True)
orders_df

,product_id,add_to_cart_order,reordered,aisle_id,department_id,reord_no.
0,49302,1.0,1.0,120,16,7.0
1,49302,7.0,1.0,120,16,7.0
2,49302,1.0,1.0,120,16,7.0
3,49302,11.0,1.0,120,16,7.0
4,49302,8.0,0.0,120,16,7.0
...,...,...,...,...,...,...
1384612,1528,12.0,0.0,97,13,0.0
1384613,47935,20.0,0.0,73,11,0.0
1384614,9491,21.0,0.0,25,11,0.0
1384615,16380,12.0,0.0,97,13,0.0


In [68]:
orders_df.drop(columns=["add_to_cart_order"], inplace=True)
orders_df

,product_id,reordered,aisle_id,department_id,reord_no.
0,49302,1.0,120,16,7.0
1,49302,1.0,120,16,7.0
2,49302,1.0,120,16,7.0
3,49302,1.0,120,16,7.0
4,49302,0.0,120,16,7.0
...,...,...,...,...,...
1384612,1528,0.0,97,13,0.0
1384613,47935,0.0,73,11,0.0
1384614,9491,0.0,25,11,0.0
1384615,16380,0.0,97,13,0.0


In [72]:
# Read Department Dataset
file_path = "../instacart-data/departments.csv"
dept_df = pd.read_csv(file_path)
dept_df.head()

,department_id,department
0,1,frozen
1,2,other
2,3,bakery
3,4,produce
4,5,alcohol


In [103]:
# Read Department Dataset
file_path = "../instacart-data/aisles.csv"
aisles_df = pd.read_csv(file_path)
aisles_df.head()

,aisle_id,aisle
0,1,prepared soups salads
1,2,specialty cheeses
2,3,energy granola bars
3,4,instant foods
4,5,marinades meat preparation


In [105]:
# Merge with department and aisle dataset
orders_df = orders_df.merge(dept_df, how="left", on="department_id")
orders_df = orders_df.merge(aisles_df, how="left", on="aisle_id")
orders_df

,product_id,reordered,aisle_id,department_id,reord_no.,department_x,department_y,department,aisle
0,49302,1.0,120,16,7.0,dairy eggs,dairy eggs,dairy eggs,yogurt
1,49302,1.0,120,16,7.0,dairy eggs,dairy eggs,dairy eggs,yogurt
2,49302,1.0,120,16,7.0,dairy eggs,dairy eggs,dairy eggs,yogurt
3,49302,1.0,120,16,7.0,dairy eggs,dairy eggs,dairy eggs,yogurt
4,49302,0.0,120,16,7.0,dairy eggs,dairy eggs,dairy eggs,yogurt
...,...,...,...,...,...,...,...,...,...
1384612,1528,0.0,97,13,0.0,pantry,pantry,pantry,baking supplies decor
1384613,47935,0.0,73,11,0.0,personal care,personal care,personal care,facial care
1384614,9491,0.0,25,11,0.0,personal care,personal care,personal care,soap
1384615,16380,0.0,97,13,0.0,pantry,pantry,pantry,baking supplies decor


In [106]:
final_df = pd.get_dummies(orders_df)
final_df

,product_id,reordered,aisle_id,department_id,reord_no.,department_x_alcohol,department_x_babies,department_x_bakery,department_x_beverages,department_x_breakfast,...,aisle_spreads,aisle_tea,aisle_tofu meat alternatives,aisle_tortillas flat bread,aisle_trail mix snack mix,aisle_trash bags liners,aisle_vitamins supplements,aisle_water seltzer sparkling water,aisle_white wines,aisle_yogurt
0,49302,1.0,120,16,7.0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
1,49302,1.0,120,16,7.0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
2,49302,1.0,120,16,7.0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
3,49302,1.0,120,16,7.0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
4,49302,0.0,120,16,7.0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1384612,1528,0.0,97,13,0.0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1384613,47935,0.0,73,11,0.0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1384614,9491,0.0,25,11,0.0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1384615,16380,0.0,97,13,0.0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [107]:
# Attempt 1
final_df.drop(columns=["product_id", "aisle_id", "department_id"], inplace=True)
final_df

,reordered,reord_no.,department_x_alcohol,department_x_babies,department_x_bakery,department_x_beverages,department_x_breakfast,department_x_bulk,department_x_canned goods,department_x_dairy eggs,...,aisle_spreads,aisle_tea,aisle_tofu meat alternatives,aisle_tortillas flat bread,aisle_trail mix snack mix,aisle_trash bags liners,aisle_vitamins supplements,aisle_water seltzer sparkling water,aisle_white wines,aisle_yogurt
0,1.0,7.0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,1
1,1.0,7.0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,1
2,1.0,7.0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,1
3,1.0,7.0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,1
4,0.0,7.0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1384612,0.0,0.0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1384613,0.0,0.0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1384614,0.0,0.0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1384615,0.0,0.0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [108]:
#Testing .. doesnt improve accuracy though

products = orders_df.product_id
reord_no = orders_df['reord_no.']
reordered = orders_df.reordered

test_df = pd.DataFrame(
           { 'product_id':products,
            'reord_no': reord_no,
            'reordered':reordered}
            )
test_df

,product_id,reord_no,reordered
0,49302,7.0,1.0
1,49302,7.0,1.0
2,49302,7.0,1.0
3,49302,7.0,1.0
4,49302,7.0,0.0
...,...,...,...
1384612,1528,0.0,0.0
1384613,47935,0.0,0.0
1384614,9491,0.0,0.0
1384615,16380,0.0,0.0


In [109]:
X = final_df.drop("reordered", axis=1)
#X = X.drop("add_to_cart_order", axis=1)
y = pd.DataFrame(final_df["reordered"])

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y)

In [110]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

X_train_scaled

array([[-0.34486793, -0.06345778, -0.10438805, ..., -0.16469239,
        -0.02783562, -0.2037105 ],
       [-0.25205976, -0.06345778, -0.10438805, ..., -0.16469239,
        -0.02783562, -0.2037105 ],
       [ 0.05143793, -0.06345778, -0.10438805, ..., -0.16469239,
        -0.02783562, -0.2037105 ],
       ...,
       [-0.36732152, -0.06345778, -0.10438805, ..., -0.16469239,
        -0.02783562, -0.2037105 ],
       [-0.40549262, -0.06345778, -0.10438805, ..., -0.16469239,
        -0.02783562, -0.2037105 ],
       [-0.40511839, -0.06345778, -0.10438805, ..., -0.16469239,
        -0.02783562, -0.2037105 ]])

In [111]:
def setup_nn_model(input_features:int, layer1:int, layer2:int, add_layers:int, 
                   actv_fun1:str, actv_func2:str, output_func:str,)->float:

    # Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
    number_input_features = input_features
    hidden_nodes_layer1 = layer1
    hidden_nodes_layer2 = layer2

    nn = tf.keras.models.Sequential()

    # First hidden layer
    nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features ,activation=actv_fun1))

    # Second hidden layer
    nn.add(
        tf.keras.layers.Dense(units=hidden_nodes_layer2, activation=actv_func2 )
    )
    while (add_layers != 0):
        nn.add(
        tf.keras.layers.Dense(units=hidden_nodes_layer2, activation=actv_func2 )
        )
        add_layers = add_layers - 1
        
    # Output layer
    nn.add(tf.keras.layers.Dense(units=1, activation=output_func))
    
    return nn


In [112]:
# Define a Training Function for the Neural Network
def train_nn(nn:float, num_epochs:int):
    # Import checkpoint dependencies
    import os
    from tensorflow.keras.callbacks import ModelCheckpoint

    # Define the checkpoint path and filenames
    os.makedirs("checkpoints/",exist_ok=True)
    checkpoint_path = "checkpoints/weights.{epoch:02d}.hdf5"

    # Create a callback that saves the model's weights every 5 epochs
    cp_callback = ModelCheckpoint(
        filepath=checkpoint_path,
        verbose=1,
        save_weights_only=True,
        save_freq=50000)
    
    # Compile the model
    nn.compile(loss="binary_crossentropy",optimizer="adam",metrics=["accuracy"])
    
    # Train the model
    fit_model = nn.fit(X_train_scaled, y_train, epochs=num_epochs, callbacks=[cp_callback])
    
    # Evaluate the model using the test data
    model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
    print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")
    
    # Export our model to HDF5 file
    nn.save("ProdReordering.h5")

In [115]:
inputs = len(X_train_scaled[0])
layer1_nodes = 3 * inputs
layer2_nodes = 2 * inputs             

# Attempt 1 - 
nn = setup_nn_model(inputs, layer1_nodes, layer2_nodes, 0, "relu", "relu", "sigmoid")

# Check the structure of the model
nn.summary()

#Train the NN
train_nn(nn, 5)

Model: "sequential_10"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_48 (Dense)             (None, 594)               118206    
_________________________________________________________________
dense_49 (Dense)             (None, 396)               235620    
_________________________________________________________________
dense_50 (Dense)             (None, 1)                 397       
Total params: 354,223
Trainable params: 354,223
Non-trainable params: 0
_________________________________________________________________


In [73]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import confusion_matrix

In [74]:
# Define the logistic regression model
log_classifier = LogisticRegression(solver="lbfgs",max_iter=200)

# Train the model
log_classifier.fit(X_train_scaled,y_train)

# Evaluate the model
y_pred = log_classifier.predict(X_test_scaled)
print(f" Logistic regression model accuracy: {accuracy_score(y_test,y_pred):.3f}")

 Logistic regression model accuracy: 0.599


In [27]:
# Use XGBoost

In [ ]:
from sklearn.svm import SVC

# Create the SVM model
svm = SVC(kernel='linear')

# Train the model
svm.fit(X_train, y_train)

# Evaluate the model
y_pred = svm.predict(X_test_scaled)
print(f" SVM model accuracy: {accuracy_score(y_test,y_pred):.3f}")